In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import os
import xarray as xr
import glob
import dask


# ESIO Imports
import esiodata as ed

In [2]:
#############################################################
# Load in Data
#############################################################

E = ed.esiodata.load()

# Load in Obs
data_dir = E.data_dir
grid_dir = E.grid_dir

nc_files = glob.glob(E.obs['NSIDC_0081']['sipn_nc']+'/*.nc')
print(len(nc_files))

1202


In [3]:
%time ds = xr.open_mfdataset(nc_files, concat_dim='time', parallel=False, autoclose=True)

CPU times: user 57.8 s, sys: 3.2 s, total: 1min 1s
Wall time: 1min


In [4]:
%time ds = xr.open_mfdataset(nc_files, concat_dim='time', parallel=True, autoclose=True)

CPU times: user 1min 16s, sys: 9.82 s, total: 1min 26s
Wall time: 1min 16s


In [5]:
from dask.distributed import Client
client = Client()
print(client)

<Client: scheduler='tcp://127.0.0.1:43291' processes=16 cores=16>


In [6]:
%time ds = xr.open_mfdataset(nc_files, concat_dim='time', parallel=True, autoclose=True)

CPU times: user 2min 17s, sys: 12.3 s, total: 2min 29s
Wall time: 3min 48s


In [ ]:

cvar = 'sic'

bag = dask.bag.from_sequence(nc_files)
load_variable = lambda ncfile: xr.open_dataset(ncfile)[cvar]
bag = bag.map(load_variable)
dataarrays = bag.compute()
%time dataarray = xr.concat(dataarrays, dim='time')

In [ ]:
# # fig_dir = os.path.join(E.fig_dir, 'obs', 'NSIDC_0081' , 'standard')
# da_51_in = xr.open_mfdataset(, concat_dim='time', autoclose=True)
# ds_ext = xr.open_dataset(os.path.join(E.obs['NSIDC_extent']['sipn_nc'], 'N_seaice_extent_daily_v3.0.nc'))
# ds_ext = ds_ext.rename({'datetime':'time'})
# print('done')